<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1">Dependencies</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2">Functions</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-3">Paths</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-4">Main</a></span><ul class="toc-item"><li><span><a href="#TRB" data-toc-modified-id="TRB-4.1">TRB</a></span><ul class="toc-item"><li><span><a href="#divstats" data-toc-modified-id="divstats-4.1.1">divstats</a></span></li><li><span><a href="#Cleanup-divstats" data-toc-modified-id="Cleanup-divstats-4.1.2">Cleanup divstats</a></span></li></ul></li><li><span><a href="#TRA" data-toc-modified-id="TRA-4.2">TRA</a></span><ul class="toc-item"><li><span><a href="#divstats" data-toc-modified-id="divstats-4.2.1">divstats</a></span></li><li><span><a href="#Cleanup-divstats" data-toc-modified-id="Cleanup-divstats-4.2.2">Cleanup divstats</a></span></li></ul></li></ul></li><li><span><a href="#Test-dataset" data-toc-modified-id="Test-dataset-5">Test dataset</a></span></li></ul></div>

# Dependencies

# Functions

In [1]:
source("/Users/anabbi/git/ped_CapTCRseq/R/Immune_diversity.R")

# Paths

In [2]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"

gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

# Main

In [3]:
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/"

## TRB

In [4]:
file_list <- list.files(h4hpath, pattern = paste0("CLONES_", "TRB"), recursive = T)

In [ ]:
tail(file_list)

In [ ]:
# remove downsample files
file_list <- file_list[!grepl("_2000000", file_list)]

Some files are duplicated. Make a table to identify and pick the larger clone file

In [ ]:
file_df <- as.data.frame(file_list)

In [ ]:
file_df$samplename <- gsub(".*CLONES_TRB", "", file_df$file_list)
file_df$samplename <- gsub(".txt", "", file_df$samplename)

In [ ]:
file_df$size <- NA
for(i in 1:nrow(file_df)){
    file_df$size[i] <- file.size(paste0(h4hpath,file_df$file_list[i]))
}

In [ ]:
head(file_df)

In [ ]:
file_df[ grepl("383-03", file_df$file_list),]

In [ ]:
file_df <- file_df[ !grepl("immunarch", file_df$file_list),]

In [ ]:
#if duplicated keep the bigger file corresponding to deeper seqeuncing.
file_df_dedup <- file_df[0,]

for(s in unique(file_df$samplename)){
    tmp <- file_df[ file_df$samplename == s,]
    if(nrow(tmp) ==1 ){
        file_df_dedup <- rbind(file_df_dedup, tmp)
    }
    if(nrow(tmp) > 1 ){
        file_df_dedup <- rbind(file_df_dedup, tmp[ tmp$size == max(tmp$size),])
    }    
}

In [ ]:
file_df_dedup[ grepl("343-05", file_df_dedup$file_list),]

In [ ]:
dim(file_df)
dim(file_df_dedup)

In [ ]:
file_df_dedup_trb <- file_df_dedup
save(file_df_dedup_trb, file = paste0(datapath,"capTCRseq/TRBfiles.RData"))

In [ ]:
batchTRB <- immunelistfx(file_df_dedup$file_list, h4hpath, "TRB")

In [ ]:
names(batchTRB) <- gsub(".*CLONES_TRB", "", names(batchTRB))

In [ ]:
head(names(batchTRB))

In [ ]:
TRBlist <- batchTRB 

In [ ]:
save(TRBlist, file = paste0(datapath,"capTCRseq/TRBlist.RData"))

### divstats

In [5]:
load(file = paste0(datapath,"capTCRseq/TRBfiles.RData"))

In [6]:
load(file = paste0(datapath,"capTCRseq/TRBlist.RData"))

In [7]:
# remove lists with no clonotype
TRBlist <- TRBlist[sapply(TRBlist, function(x) length((x)) != 0)]

In [8]:
length(names(TRBlist))

[1] 580

In [9]:
# add filenames to divstats
myfilenames <- file_df_dedup_trb[ file_df_dedup_trb$samplename %in% names(TRBlist),]

In [ ]:
Divstats.fx(TRBlist, "TRB", "CHP" , paste0(datapath, "capTCRseq/"))

### Cleanup divstats

In [10]:
divstats <- read.csv(paste0(datapath, "capTCRseq/divstats_TRBCHP.csv"), header = T, stringsAsFactors = F, row.names = 1)

In [11]:
head(rownames(divstats))

[1] "CHP_315-05-PBMC-DNA" "CHP_350-01-PBMC-DNA" "CHP_350-02-PBMC-DNA"
[4] "CHP_350-03-PBMC-DNA" "CHP_350-04-PBMC-DNA" "CHP_353-02-PBMC-DNA"

In [12]:
head(divstats)

,TRB,Reads,CPKR,Average_reads,VMR,Max_reads,Singletons,Doubletons,qD,Sample_Coverage,⋯,SE_Richeness,observed_Shannon,estimated_Shannon,SE_Shannon,observed_Simpson,estimated_Simpson,SE_Simpson,sample_id,file,basename
,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
CHP_315-05-PBMC-DNA,27,659,40.97117,24.407407,1.4380040,187,1,1,27.432,1.000,⋯,1.321,15.470,15.793,0.629,9.026,9.138,0.749,CHP_315_05,batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315-05-PBMC-DNA
CHP_350-01-PBMC-DNA,153,1647,92.89617,10.764706,0.7872963,59,5,6,154.893,1.000,⋯,2.505,119.726,125.659,2.301,94.691,100.406,3.548,CHP_350_01,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-01-PBMC-DNA.txt,CHP_350-01-PBMC-DNA
CHP_350-02-PBMC-DNA,269,2992,89.90642,11.122677,0.8300771,92,17,5,281.850,0.997,⋯,19.804,208.211,219.073,2.860,159.505,168.431,5.211,CHP_350_02,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-02-PBMC-DNA.txt,CHP_350-02-PBMC-DNA
CHP_350-03-PBMC-DNA,207,2134,97.00094,10.309179,0.7807083,52,9,10,210.609,1.000,⋯,3.601,159.579,167.953,2.785,128.847,137.062,3.485,CHP_350_03,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-03-PBMC-DNA.txt,CHP_350-03-PBMC-DNA
CHP_350-04-PBMC-DNA,106,989,107.17897,9.330189,0.7731222,34,10,2,114.239,0.993,⋯,24.214,80.930,86.422,2.147,66.580,71.313,2.548,CHP_350_04,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-04-PBMC-DNA.txt,CHP_350-04-PBMC-DNA
CHP_353-02-PBMC-DNA,16,234,68.37607,14.625000,0.5897147,35,1,0,16.000,1.000,⋯,0.545,13.360,13.807,0.503,12.066,12.668,0.655,CHP_353_02,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_353-02-PBMC-DNA.txt,CHP_353-02-PBMC-DNA


In [13]:
#add 9 samples with nonproductive cdr3
addsamples <- c('CHP_347-01-cfDNA','CHP_366-05_cfDNA','CHP_377-01-cfDNA',
                'CHP_381-05_cfDNA','CHP_391-02_cfDNA','CHP_392-01_cfDNA',
                'CHP_398-01_cfDNA','CHP_401-03_cfDNA','CHP_419-FFPE-DNA_S2')

In [14]:
divstats[ nrow(divstats) + 9 , ] <- NA

In [17]:
rownames(divstats)[is.na(divstats$TRB)] <- addsamples

In [18]:
#example sample_id CHP_xxx_xx
divstats$sample_id <- rownames(divstats)

#example basename CHP_xxx-xx-PBMC-DNA
divstats$basename <- rownames(divstats)

In [19]:
# add filenames to divstats
divstats$file <- myfilenames$file_list[ match(divstats$sample_id, myfilenames$samplename)]

Separate samples

In [21]:
#divstats$Sample <- NULL

In [22]:
divstats$Sample <- NA
divstats$Sample[grepl("PBMC", rownames(divstats))] <- "PBMC"
divstats$Sample[grepl("cfDNA", rownames(divstats))] <- "cfDNA"
divstats$Sample[grepl("FFPE", rownames(divstats))] <- "Tumor"

Separate LFS and non-LFS

In [23]:
divstats$Group <- NA
divstats$Group[grepl("LFS", rownames(divstats))] <- "LFS"
divstats$Group[grepl("CHP", rownames(divstats))] <- "CHP"

In [24]:
#table(divstats$Group, useNA = "always")

In [25]:
#tapply(divstats$TRB, divstats$sampletype, summary)

In [26]:
summary(divstats$Reads)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max.     NA's 
     1.0    841.8   3088.5  18710.4  12587.8 363060.0        9 

Get patient ids

In [27]:
divstats$Patient <- NA
# CHP
divstats$Patient[divstats$Group == "CHP"] <- substr(rownames(divstats)[divstats$Group == "CHP"],1,7)
# Normal samples have either YO or MO
divstats$Patient[grepl("YO", rownames(divstats))] <- gsub("_PBMC_DNA", "", rownames(divstats)[grepl("YO", rownames(divstats))])
divstats$Patient[grepl("MO", rownames(divstats))] <- gsub("_PBMC_DNA", "", rownames(divstats)[grepl("MO", rownames(divstats))])
# LFS
divstats$Patient[divstats$Group == "LFS"] <- substr(rownames(divstats)[divstats$Group == "LFS"],1,12)

Get cycles. For LFS, Normals and Tumors recode to 00

In [28]:
divstats$Cycle <- NA
divstats$Cycle[divstats$Group == "CHP"] <- substr(rownames(divstats)[divstats$Group == "CHP"],9,10)
divstats$Cycle[grepl("YO", rownames(divstats))] <- "00"
divstats$Cycle[grepl("MO", rownames(divstats))] <- "00"
divstats$Cycle[ divstats$Sample == "Tumor"] <- "00"
divstats$Cycle[ divstats$Group == "LFS"] <- "00"

In [29]:
divstats$sample_id <- NA

# bind patient and cycle for PBMC and cfdna to generate sample id
divstats$sample_id[ divstats$Sample != "Tumor"] <- paste(divstats$Patient[ divstats$Sample != "Tumor"], "_", 
                                                          divstats$Cycle[ divstats$Sample != "Tumor"], sep = "")

# For tumor samples start with rownames because some of them are two biopsies from the same tumor
divstats$sample_id[ divstats$Sample == "Tumor"] <- gsub("-FFPE-DN.*", "", rownames(divstats)[ divstats$Sample == "Tumor"])

In [30]:
divstats$sample_id <- gsub("-", "_", divstats$sample_id)

In [31]:
divstats$Patient <- NULL
divstats$Cycle <- NULL
divstats$Sample <- NULL
divstats$Group <- NULL

In [35]:
tail(divstats,10)

,TRB,Reads,CPKR,Average_reads,VMR,Max_reads,Singletons,Doubletons,qD,Sample_Coverage,⋯,SE_Richeness,observed_Shannon,estimated_Shannon,SE_Shannon,observed_Simpson,estimated_Simpson,SE_Simpson,sample_id,file,basename
,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
CHP_383-03_cfDNA,2,1135,1.762115,567.5,1.117665,1016,0,0,2,1,⋯,0,1.399,1.399,0.028,1.231,1.231,0.022,CHP_383_03,remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRBCHP_383-03_cfDNA.txt,CHP_383-03_cfDNA
CHP_347-01-cfDNA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,CHP_347_01,NA,CHP_347-01-cfDNA
CHP_366-05_cfDNA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,CHP_366_05,NA,CHP_366-05_cfDNA
CHP_377-01-cfDNA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,CHP_377_01,NA,CHP_377-01-cfDNA
CHP_381-05_cfDNA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,CHP_381_05,NA,CHP_381-05_cfDNA
CHP_391-02_cfDNA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,CHP_391_02,NA,CHP_391-02_cfDNA
CHP_392-01_cfDNA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,CHP_392_01,NA,CHP_392-01_cfDNA
CHP_398-01_cfDNA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,CHP_398_01,NA,CHP_398-01_cfDNA
CHP_401-03_cfDNA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,CHP_401_03,NA,CHP_401-03_cfDNA


In [36]:
write.csv(divstats,
          paste0(datapath, "capTCRseq/divstats_TRBCHP.csv"), row.names = TRUE)

## TRA

In [42]:
file_list <- list.files(h4hpath, pattern = paste0("CLONES_", "TRA"), recursive = T)

In [43]:
tail(file_list)

[1] "remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRACHP_365-03_cfDNA.txt"
[2] "remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRACHP_370-02_cfDNA.txt"
[3] "remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRACHP_374-03_cfDNA.txt"
[4] "remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRACHP_378-01_cfDNA.txt"
[5] "remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRACHP_381-04_cfDNA.txt"
[6] "remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRACHP_383-03_cfDNA.txt"

In [44]:
# remove downsample files
file_list <- file_list[!grepl("_2000000", file_list)]

Some files are duplicated. Make a table to identify and pick the larger clone file

In [45]:
file_df <- as.data.frame(file_list)

In [46]:
file_df$samplename <- gsub(".*CLONES_TRA", "", file_df$file_list)
file_df$samplename <- gsub(".txt", "", file_df$samplename)

In [47]:
file_df$size <- NA
for(i in 1:nrow(file_df)){
    file_df$size[i] <- file.size(paste0(h4hpath,file_df$file_list[i]))
}

In [48]:
head(file_df)

,file_list,samplename,size
,<chr>,<chr>,<dbl>
1,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRACHP_315-05-PBMC-DNA.txt,CHP_315-05-PBMC-DNA,31086
2,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRACHP_350-01-PBMC-DNA.txt,CHP_350-01-PBMC-DNA,490223
3,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRACHP_350-02-PBMC-DNA.txt,CHP_350-02-PBMC-DNA,707406
4,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRACHP_350-03-PBMC-DNA.txt,CHP_350-03-PBMC-DNA,605922
5,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRACHP_350-04-PBMC-DNA.txt,CHP_350-04-PBMC-DNA,273004
6,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRACHP_353-02-PBMC-DNA.txt,CHP_353-02-PBMC-DNA,38414


In [49]:
file_df[ grepl("383-03", file_df$file_list),]

,file_list,samplename,size
,<chr>,<chr>,<dbl>
379,batch8_14Jan2020/mixcr/clones/CLONES_TRACHP_383-03-PBMC-DNA.txt,CHP_383-03-PBMC-DNA,5798
619,remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRACHP_383-03_cfDNA.txt,CHP_383-03_cfDNA,12341


In [50]:
file_df <- file_df[ !grepl("immunarch", file_df$file_list),]

In [51]:
file_df_dedup <- file_df[0,]

for(s in unique(file_df$samplename)){
    tmp <- file_df[ file_df$samplename == s,]
    if(nrow(tmp) ==1 ){
        file_df_dedup <- rbind(file_df_dedup, tmp)
    }
    if(nrow(tmp) > 1 ){
        file_df_dedup <- rbind(file_df_dedup, tmp[ tmp$size == max(tmp$size),])
    }    
}

In [52]:
file_df_dedup[ grepl("343-05", file_df_dedup$file_list),]

,file_list,samplename,size
,<chr>,<chr>,<dbl>
207,batch2_Aug62019/mixcr/clones/CLONES_TRACHP_343-05-PBMC-DNA.txt,CHP_343-05-PBMC-DNA,51477
560,remaining_batch_3_cfDNA_15April2021/mixcr/clones/CLONES_TRACHP_343-05-cfDNA.txt,CHP_343-05-cfDNA,8357


In [53]:
file_df_dedup_tra <- file_df_dedup
save(file_df_dedup_tra, file = paste0(datapath,"capTCRseq/TRAfiles.RData"))

In [54]:
dim(file_df)
dim(file_df_dedup)

[1] 619   3

[1] 603   3

In [17]:
batchTRA <- immunelistfx(file_df_dedup$file_list, h4hpath, "TRA")

number of clones in file:batch4_19Aug2019/mixcr/clones/CLONES_TRACHP_315-05-PBMC-DNA.txt



[1] 106


number of non prodcutive CDR3s:



[1] 39


nonproductive aaCDR3 removed

number of clones in file:batch_reseq_24Feb2020/mixcr/clones/CLONES_TRACHP_350-01-PBMC-DNA.txt



...

nonproductive aaCDR3 removed

number of clones in file:remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRACHP_383-03_cfDNA.txt



[1] 38


number of non prodcutive CDR3s:



[1] 21


nonproductive aaCDR3 removed



In [18]:
names(batchTRA) <- gsub(".*CLONES_TRA", "", names(batchTRA))

In [19]:
TRAlist <- batchTRA

In [20]:
save(TRAlist, file = paste0(datapath,"capTCRseq/TRAlist.RData"))

### divstats

In [76]:
load(file = paste0(datapath,"capTCRseq/TRAfiles.RData"))

In [77]:
load(file = paste0(datapath,"capTCRseq/TRAlist.RData"))

In [78]:
# remove lists with no clonotype
TRAlist <- TRAlist[sapply(TRAlist, function(x) length((x)) != 0)]

In [79]:
length(TRAlist)

[1] 596

In [24]:
Divstats.fx(TRAlist, "TRA", "CHP" , paste0(datapath, "capTCRseq/"))

Warning message in BootstrapFun.abun(x = x, FunName, datatype, B):
“The Bootstrap community has only one species. Estimation is not robust.”
Warning message in BootstrapFun.abun(x = x, FunName, datatype, B):
“The Bootstrap community has only one species. Estimation is not robust.”
Warning message in BootstrapFun.abun(x = x, FunName, datatype, B):
“The Bootstrap community has only one species. Estimation is not robust.”
Warning message in BootstrapFun.abun(x = x, FunName, datatype, B):
“The Bootstrap community has only one species. Estimation is not robust.”
Warning message in BootstrapFun.abun(x = x, FunName, datatype, B):
“The Bootstrap community has only one species. Estimation is not robust.”
Warning message in BootstrapFun.abun(x = x, FunName, datatype, B):
“The Bootstrap community has only one species. Estimation is not robust.”
Warning message in BootstrapFun.abun(x = x, FunName, datatype, B):
“The Bootstrap community has only one species. Estimation is not robust.”
Warning messa

### Cleanup divstats

In [80]:
divstats <- read.csv(paste0(datapath, "capTCRseq/divstats_TRACHP.csv"), header = T, stringsAsFactors = F, row.names = 1)

In [81]:
# add filenames to divstats
myfilenames <- file_df_dedup_tra[ file_df_dedup_tra$samplename %in% names(TRAlist),]

In [82]:
dim(divstats)

[1] 596  22

In [83]:
head(divstats)

,TRA,Reads,CPKR,Average_reads,VMR,Max_reads,Singletons,Doubletons,qD,Sample_Coverage,⋯,SE_Richeness,observed_Shannon,estimated_Shannon,SE_Shannon,observed_Simpson,estimated_Simpson,SE_Simpson,sample_id,file,basename
,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
CHP_315-05-PBMC-DNA,67,1636,40.95355,24.41791,0.9377717,107,2,0,67.864,1.000,⋯,2.331,47.431,48.435,1.015,35.900,36.683,1.210,NA_NA,batch4_19Aug2019/mixcr/clones/CLONES_TRACHP_315-05-PBMC-DNA.txt,CHP_315-05-PBMC-DNA
CHP_350-01-PBMC-DNA,641,13038,49.16398,20.34009,0.7859894,116,40,10,672.477,0.998,⋯,36.875,490.352,504.229,3.087,396.458,408.861,4.877,NA_NA,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRACHP_350-01-PBMC-DNA.txt,CHP_350-01-PBMC-DNA
CHP_350-02-PBMC-DNA,908,18934,47.95606,20.85242,0.7266378,120,41,15,937.076,0.999,⋯,23.908,717.297,736.184,3.418,594.466,613.703,5.500,NA_NA,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRACHP_350-02-PBMC-DNA.txt,CHP_350-02-PBMC-DNA
CHP_350-03-PBMC-DNA,755,13957,54.09472,18.48609,0.7250201,103,46,9,793.067,0.998,⋯,53.426,594.801,613.585,3.273,495.096,513.267,5.431,NA_NA,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRACHP_350-03-PBMC-DNA.txt,CHP_350-03-PBMC-DNA
CHP_350-04-PBMC-DNA,381,6749,56.45281,17.71391,0.6824403,72,24,2,403.106,0.997,⋯,118.169,304.600,315.309,2.568,260.157,270.547,3.631,NA_NA,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRACHP_350-04-PBMC-DNA.txt,CHP_350-04-PBMC-DNA
CHP_353-02-PBMC-DNA,56,1404,39.88604,25.07143,0.7001978,92,3,1,58.189,0.999,⋯,7.189,44.969,45.951,0.710,37.799,38.817,1.195,NA_NA,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRACHP_353-02-PBMC-DNA.txt,CHP_353-02-PBMC-DNA


In [84]:
divstats$sample_id <- rownames(divstats)
divstats$basename <- rownames(divstats)

In [85]:
# add filenames to divstats
divstats$file <- myfilenames$file_list[ match(divstats$sample_id, myfilenames$samplename)]

In [86]:
dim(divstats)

[1] 596  22

Separate samples

In [87]:
divstats$Sample <- NA
divstats$Sample[grepl("PBMC", rownames(divstats))] <- "PBMC"
divstats$Sample[grepl("cfDNA", rownames(divstats))] <- "cfDNA"
divstats$Sample[grepl("FFPE", rownames(divstats))] <- "Tumor"

Separate LFS and non-LFS

In [88]:
divstats$Group <- NA
divstats$Group[grepl("LFS", rownames(divstats))] <- "LFS"
divstats$Group[grepl("CHP", rownames(divstats))] <- "CHP"

Get patient ids

In [89]:
divstats$Patient <- NA
# CHP
divstats$Patient[divstats$Group == "CHP"] <- substr(rownames(divstats)[divstats$Group == "CHP"],1,7)
# Normal samples have either YO or MO
divstats$Patient[grepl("YO", rownames(divstats))] <- gsub("_PBMC_DNA", "", rownames(divstats)[grepl("YO", rownames(divstats))])
divstats$Patient[grepl("MO", rownames(divstats))] <- gsub("_PBMC_DNA", "", rownames(divstats)[grepl("MO", rownames(divstats))])
# LFS
divstats$Patient[divstats$Group == "LFS"] <- substr(rownames(divstats)[divstats$Group == "LFS"],1,12)

Get cycles. For LFS, Normals and Tumors recode to 00

In [90]:
divstats$Cycle <- NA
divstats$Cycle[divstats$Group == "CHP"] <- substr(rownames(divstats)[divstats$Group == "CHP"],9,10)

divstats$Cycle[grepl("YO", rownames(divstats))] <- "00"
divstats$Cycle[grepl("MO", rownames(divstats))] <- "00"
divstats$Cycle[ divstats$Sample == "Tumor"] <- "00"
divstats$Cycle[ divstats$Group == "LFS"] <- "00"

In [91]:
divstats$sample_id <- NA

# bind patient and cycle for PBMC and cfdna to generate sample id
divstats$sample_id[ divstats$Sample != "Tumor"] <- paste(divstats$Patient[ divstats$Sample != "Tumor"], "_", 
                                                          divstats$Cycle[ divstats$Sample != "Tumor"], sep = "")

# For tumor samples start with rownames because some of them are two biopsies from the same tumor
divstats$sample_id[ divstats$Sample == "Tumor"] <- gsub("-FFPE-DN.*", "", rownames(divstats)[ divstats$Sample == "Tumor"])

In [92]:
divstats$sample_id <- gsub("-", "_", divstats$sample_id)

In [43]:
divstats$Patient <- NULL
divstats$Cycle <- NULL
divstats$Group <- NULL
divstats$Sample <- NULL

In [93]:
head(divstats)

,TRA,Reads,CPKR,Average_reads,VMR,Max_reads,Singletons,Doubletons,qD,Sample_Coverage,⋯,observed_Simpson,estimated_Simpson,SE_Simpson,sample_id,file,basename,Sample,Group,Patient,Cycle
,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
CHP_315-05-PBMC-DNA,67,1636,40.95355,24.41791,0.9377717,107,2,0,67.864,1.000,⋯,35.900,36.683,1.210,CHP_315_05,batch4_19Aug2019/mixcr/clones/CLONES_TRACHP_315-05-PBMC-DNA.txt,CHP_315-05-PBMC-DNA,PBMC,CHP,CHP_315,05
CHP_350-01-PBMC-DNA,641,13038,49.16398,20.34009,0.7859894,116,40,10,672.477,0.998,⋯,396.458,408.861,4.877,CHP_350_01,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRACHP_350-01-PBMC-DNA.txt,CHP_350-01-PBMC-DNA,PBMC,CHP,CHP_350,01
CHP_350-02-PBMC-DNA,908,18934,47.95606,20.85242,0.7266378,120,41,15,937.076,0.999,⋯,594.466,613.703,5.500,CHP_350_02,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRACHP_350-02-PBMC-DNA.txt,CHP_350-02-PBMC-DNA,PBMC,CHP,CHP_350,02
CHP_350-03-PBMC-DNA,755,13957,54.09472,18.48609,0.7250201,103,46,9,793.067,0.998,⋯,495.096,513.267,5.431,CHP_350_03,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRACHP_350-03-PBMC-DNA.txt,CHP_350-03-PBMC-DNA,PBMC,CHP,CHP_350,03
CHP_350-04-PBMC-DNA,381,6749,56.45281,17.71391,0.6824403,72,24,2,403.106,0.997,⋯,260.157,270.547,3.631,CHP_350_04,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRACHP_350-04-PBMC-DNA.txt,CHP_350-04-PBMC-DNA,PBMC,CHP,CHP_350,04
CHP_353-02-PBMC-DNA,56,1404,39.88604,25.07143,0.7001978,92,3,1,58.189,0.999,⋯,37.799,38.817,1.195,CHP_353_02,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRACHP_353-02-PBMC-DNA.txt,CHP_353-02-PBMC-DNA,PBMC,CHP,CHP_353,02


In [94]:
write.csv(divstats,
          paste0(datapath, "capTCRseq/divstats_TRACHP.csv"), row.names = TRUE)

# Test dataset

In [4]:
mordpath <- "/Users/anabbi/Desktop/Sam/immpedcan/mixcr/Tcelllines/muldercelllines/mixcr/clones/"

In [5]:
file_list <- list.files(mordpath, pattern = paste0("CLONES_", "TRB"), recursive = T)

In [6]:
file_list

[1] "CLONES_TRBA037_S1.txt"           "CLONES_TRBL2D8_S2.txt"          
 [3] "CLONES_TRBM36-EZM_S4.txt"        "CLONES_TRBM36-TIL2-1E7P3_S3.txt"
 [5] "CLONES_TRBOV7-TIL2_S5.txt"       "CLONES_TRBSE14-2005_S7.txt"     
 [7] "CLONES_TRBSE14-2033_S8.txt"      "CLONES_TRBSE14-2034_S9.txt"     
 [9] "CLONES_TRBSE14-2035_S10.txt"     "CLONES_TRBSTIM1-CY3_S6.txt"     
[11] "CLONES_TRBUndetermined_S0.txt"

In [7]:
file_list <- file_list[ file_list != 'CLONES_TRBUndetermined_S0.txt']

In [8]:
file_list

[1] "CLONES_TRBA037_S1.txt"           "CLONES_TRBL2D8_S2.txt"          
 [3] "CLONES_TRBM36-EZM_S4.txt"        "CLONES_TRBM36-TIL2-1E7P3_S3.txt"
 [5] "CLONES_TRBOV7-TIL2_S5.txt"       "CLONES_TRBSE14-2005_S7.txt"     
 [7] "CLONES_TRBSE14-2033_S8.txt"      "CLONES_TRBSE14-2034_S9.txt"     
 [9] "CLONES_TRBSE14-2035_S10.txt"     "CLONES_TRBSTIM1-CY3_S6.txt"

In [9]:
cells <- immunelistfx(file_list, mordpath, "TRB")

number of clones in file:CLONES_TRBA037_S1.txt



[1] 106


number of non prodcutive CDR3s:



[1] 22


nonproductive aaCDR3 removed

number of clones in file:CLONES_TRBL2D8_S2.txt



...

nonproductive aaCDR3 removed

number of clones in file:CLONES_TRBSTIM1-CY3_S6.txt



[1] 38


number of non prodcutive CDR3s:



[1] 10


nonproductive aaCDR3 removed



In [10]:
cells

$A037_S1
 [1] 3 3 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[39] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[77] 1 1 1 1 1 1 1 1

$L2D8_S2
[1] 281

$`M36-EZM_S4`
[1] 2 2 2 1 1 1 1 1

$`M36-TIL2-1E7P3_S3`
 [1] 25 23 18 16 15 12 11 10 10 10  8  7  7  6  6  6  5  5  4  4  4  4  4  3  3
[26]  3  3  3  3  3  3  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  1  1
[51]  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1

$`OV7-TIL2_S5`
 [1] 177  58  12  11   8   8   8   4   4   4   4   3   2   2   2   1   1   1   1
[20]   1   1   1   1   1   1   1   1   1   1   1   1   1

$`SE14-2005_S7`
[1] 331   3   1

$`SE14-2033_S8`
[1] 264

$`SE14-2034_S9`
[1] 471

$`SE14-2035_S10`
[1] 460

$`STIM1-CY3_S6`
 [1] 124  76  48  34  25  11  10   9   9   8   4   3   3   2   2   2   2   2   2
[20]   1   1   1   1   1   1   1   1   1

In [11]:
myimmunelist_cells <- cells 

In [12]:
save(myimmunelist_cells, file = paste0(datapath,"capTCRseq/myimmunelist_cells.RData"))

In [13]:
Divstats.fx(myimmunelist_cells, "TRB", "Cells" , paste0(datapath, "capTCRseq/"))

Warning message in BootstrapFun.abun(x = x, FunName, datatype, B):
“The Bootstrap community has only one species. Estimation is not robust.”
Warning message in BootstrapFun.abun(x = x, FunName, datatype, B):
“The Bootstrap community has only one species. Estimation is not robust.”
Warning message in BootstrapFun.abun(x = x, FunName, datatype, B):
“The Bootstrap community has only one species. Estimation is not robust.”
Warning message in BootstrapFun.abun(x = x, FunName, datatype, B):
“The Bootstrap community has only one species. Estimation is not robust.”
Warning message in max(x[-1] > 1):
“no non-missing arguments to max; returning -Inf”
Warning message in BootstrapFun.abun(x = x, FunName, datatype, B):
“The Bootstrap community has only one species. Estimation is not robust.”
Warning message in max(x[-1] > 1):
“no non-missing arguments to max; returning -Inf”
Warning message in BootstrapFun.abun(x = x, FunName, datatype, B):
“The Bootstrap community has only one species. Estimation 